##### Mise en forme des dossiers de travail

In [1]:
!python --version

Python 3.7.16


In [2]:
import os
if os.getcwd().split('\\')[-1]!='OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation':
    # si valeur diff 'C:\\Users\\...\\Projets\\P6\\OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation'
    os.chdir('OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation')

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#import tensorflow_core as tfc
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))#tf.config.list_physical_devices('GPU')
print('la version de tensorflow est: ', tf.__version__)
#print('la version de tensorflow_core est: ', tfc.__version__)

In [ ]:
import pandas as pd
df = pd.read_csv('data/flipkart_com-ecommerce_sample_1050.csv')   

df['cat_1'] = df.product_category_tree.str.replace('\\[|\\"','',regex=True).str.split(" >> ",expand = True)[0]\
.apply(lambda x:x.replace(' ','_'))

In [ ]:
IMAGE_SIZE = (224,224)
batch_size = 32
epochs = 75

train_path = 'data/cleaned/Images/train'
test_path  = 'data/cleaned/Images/test'
all_path   = 'data/cleaned/Images/all'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

if not os.path.exists( train_path ): 
    os.makedirs( train_path )
    
if not os.path.exists( test_path ): 
    os.makedirs( test_path )

for repertoire in df.cat_1.unique():
    
    Xdf = df.loc[df.cat_1==repertoire]
    #print(df.iloc[i][['cat_1']].values[0])
    
    # Create the repositorys only if it doesn't existe yet
    for t in [train_path, test_path,all_path]:

        if not os.path.exists( t+'/'+"_".join(repertoire.split(' ')) ): 
            os.makedirs( t+'/'+"_".join(repertoire.split(' ')) )
    
    # create train test split
    X_train, X_test, y_train, y_test = train_test_split( Xdf[['image']], Xdf[['cat_1']], test_size=0.33, random_state=42)
    
    # resize and write .jpg train files
    for img in X_train.values:  
        image = Image.open('data/source/raw_folder_image/'+img[0]) 

        # Resizing image
        new_image = image.resize((224, 224))

        # Sauvegarde image format JPEG
        new_image.save(train_path+'/'+"_".join(repertoire.split(' '))+'/'+img[0],format = 'JPEG')
    
    # resize and write .jpg test files
    for img in X_test.values:  
        image = Image.open('data/source/raw_folder_image/'+img[0]) 

        # Resizing image
        new_image = image.resize((224, 224))

        # Sauvegarde image format JPEG
        new_image.save(test_path+'/'+"_".join(repertoire.split(' '))+'/'+img[0],format = 'JPEG')

        

## Utilisation du VGG-16 pré-entraîné

## Transfert Learning

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras import Model
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [ ]:
VGG16(weights="imagenet", input_shape=(224, 224, 3)).summary()

In [ ]:
model.summary()

In [ ]:
# On entraîne seulement le nouveau classifieur et on ne ré-entraîne pas les autres couches :
for layer in model.layers:
   layer.trainable = False

In [ ]:
# import glob2
train_folders = glob2.glob(train_path+'/*')

# Récupérer la sortie de ce réseau et ajout d'une couche de Flatten 
flat1 = Flatten()(model.output)

class1 = Dense(1024, activation='relu')(flat1)
class2 = Dense(1024, activation='relu')(class1)

# Ajouter la nouvelle couche fully-connected pour la classification à 7 classes
predictions = Dense(len(train_folders), activation='softmax')(class2)

# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

In [ ]:
new_model.summary()

In [ ]:
# Compiler le modèle 
from keras import optimizers
adam = optimizers.Adam()

new_model.compile(loss="categorical_crossentropy", 
                  optimizer= adam, #optimizers.SGD(lr=0.0001, momentum=0.9), # adam
                  metrics=["accuracy"])




## Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


train_image_generator = ImageDataGenerator(
    rescale=1./255)

test_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           color_mode = "rgb",
                                                           directory= train_path,
                                                           shuffle=True,
                                                           target_size=IMAGE_SIZE, 
                                                           class_mode='categorical',
                                                           save_to_dir='data/cleaned/Images/augmented images train/',
                                                           save_prefix='augm_',
                                                            save_format='jpg',
                                                          )

val_data_gen = test_image_generator.flow_from_directory(batch_size = batch_size, 
                                                              directory = test_path, 
                                                              color_mode = 'rgb',
                                                              target_size=IMAGE_SIZE,
                                                              class_mode='categorical')


## Entrainement

In [ ]:
%%time
new_model.fit_generator(train_data_gen, 
                    epochs=epochs,  # one forward/backward pass of training data
                    steps_per_epoch=700 //batch_size,  # number of images comprising of one epoch
                    validation_data=val_data_gen, # Or validation_data=valid_generator
                    # validation_steps= 350 //batch_size
                    verbose = 2
                       )


In [ ]:
import matplotlib.pyplot as plt
plt.plot(new_model.history.history['accuracy'],label='train')
plt.plot(new_model.history.history['val_accuracy'],label='test')
plt.title('CNN Model accuracy values')
plt.xlabel('Epoch')
plt.legend(loc='upper right')